# NasNet Dogs v Cats

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.conv_learner import *
PATH = "data/dogscats/"
sz=224; bs=48

In [3]:
def nasnet(pre): return nasnetalarge(pretrained = 'imagenet' if pre else None)
model_features[nasnet]=4032*2

In [4]:
stats = ([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
tfms = tfms_from_stats(stats, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

#Source:   
#def tfms_from_stats(stats, sz, aug_tfms=None, max_zoom=None, pad=0, crop_type=None, tfm_y=None):
#    if aug_tfms is None: aug_tfms=[]
#    tfm_norm = Normalize(*stats)
#    tfm_denorm = Denormalize(*stats)
#    val_tfm = image_gen(tfm_norm, tfm_denorm, sz, pad=pad, crop_type=CropType.CENTER, tfm_y=tfm_y)
#    trn_tfm=image_gen(tfm_norm, tfm_denorm, sz, tfms=aug_tfms, max_zoom=max_zoom,
#                      pad=pad, crop_type=crop_type, tfm_y=tfm_y)
#    return trn_tfm, val_tfm
#File:      ~/fastai/courses/dl1/fastai/transforms.py

In [5]:
# http://forums.fast.ai/t/nasnet-py-changes/9344/6
# I have to do several modifications to get this code worked.
# 1. change model url (L10, L19)
#        http://webia.lip6.fr/~cadene/Downloads/pretrained-models.pytorch/nasnetalarge-dc8c1432.pth
#        =>
#        http://data.lip6.fr/cadene/pretrainedmodels/nasnetalarge-a1897284.pth
# 2. change self.linear to self.last_linear (L548)
#       self.linear = nn.Linear(4032, self.num_classes)
#       =>
#       self.last_linear = nn.Linear(4032, self.num_classes)
# 3. change model.linear to model.last_linear, new_linear to new_last_linear (L607-610)
#       new_linear = nn.Linear(model.linear.in_features, 1000)
#       new_linear.weight.data = model.linear.weight.data[1:]
#       new_linear.bias.data = model.linear.bias.data[1:]
#       model.linear = new_linear
#       =>
#       new_last_linear = nn.Linear(model.last_linear.in_features, 1000)
#       new_last_linear.weight.data = model.last_linear.weight.data[1:]
#       new_last_linear.bias.data = model.last_linear.bias.data[1:]
#       model.last_linear = new_last_linear

learn = ConvLearner.pretrained(nasnet, data, precompute=True, xtra_fc=[], ps=0.5)

In [6]:
%time learn.fit(1e-2, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.075737   0.071858   0.990079  
    1      0.07328    0.062094   0.991567                     

CPU times: user 11.6 s, sys: 11.5 s, total: 23.1 s
Wall time: 16.6 s


[0.06209377, 0.991567462682724]

In [7]:
learn.precompute=False
learn.bn_freeze=True

In [8]:
%time learn.fit(1e-2, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.069971   0.062008   0.988591  

CPU times: user 12min 38s, sys: 4min 51s, total: 17min 30s
Wall time: 15min 13s


[0.062007807, 0.9885912722065335]

In [9]:
learn.save('nas_pre')

In [10]:
def freeze_to(m, n):
    c=children(m[0])
    for l in c:     set_trainable(l, False)
    for l in c[n:]: set_trainable(l, True)

In [11]:
freeze_to(learn.model, 17)

In [12]:
learn.fit([1e-5,1e-4,1e-2], 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                     
    0      0.093285   0.0743     0.988095  



[0.074300304, 0.9880952423527127]

In [13]:
learn.save('nas')